## Вопрос №1

На языке Python написать алгоритм (функцию) определения четности целого числа, который будет аналогичен нижеприведенному по функциональности, но отличен по своей сути. Объяснить плюсы и минусы обеих реализаций. 
Пример: 
```
def isEven(value):
    return value % 2 == 0
```

## Вопрос №2

На языке Python написать минимум по 2 класса реализовывающих циклический буфер FIFO. Объяснить плюсы и минусы каждой реализации.
Оценивается:
* Полнота и качество реализации
* Оформление кода
* Наличие сравнения и пояснения по быстродействию

## Вопрос №3

На языке Python предложить алгоритм, который быстрее всего (по процессорным тикам) отсортирует данный ей массив чисел. Массив может быть любого размера со случайным порядком чисел (в том числе и отсортированным). Объяснить, почему вы считаете, что функция соответствует заданным критериям.

# Вопрос №1

In [11]:
def is_even(value):
    return not value & 1

In [15]:
def isEven(value):
    return value % 2 == 0

In [17]:
from timeit import timeit

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

print(timeit(lambda: [isEven(x) for x in numbers], number=10000))
print(timeit(lambda: [is_even(x) for x in numbers], number=10000))


0.005946299999777693
0.005567400000018097


## Объяснение

**Наглядно показано, что метод, использующий побитовый оператор для проверки четности работает немного быстрее метода, использующего оператор деления с остатком.**

Минусом реализации с побитовым оператором является ее не лучшая читаемость. Некоторым людям может быть не понятно что делает функция по ее реализации.

# Вопрос№2

In [83]:
class FIFO1:
    def __init__(self, values=None):
        if values is None:
            self.values = []
        else:
            self.values = values[::]
            
    def pop(self):
        if len(self.values) > 0:
            return self.values.pop(0)
        else:
            return None
    
    def push(self, value):
        self.values.append(value)
        
    def __repr__(self):
        return repr(self.values)


In [84]:
class FIFO2:
    class Element:
        def __init__(self, value, _next=None, _prev=None):
            self.value = value
            self.next = _next
            self.prev = _prev
    
    def __init__(self, values=None):
        self.head = None
        self.tail = None
        if values is not None and len(values) > 0:
            for value in values:
                self.push(value)

    def pop(self):
        if self.tail is None:
            return None
        tail_value = self.tail.value
        if self.tail.prev is not None:
            self.tail = self.tail.prev
            self.tail.next = None
        else:
            self.head = None
            self.tail = None
        return tail_value
        
    def push(self, value):
        if self.head is None:
            self.head = FIFO2.Element(value)
            self.tail = self.head
        else:
            new_head = FIFO2.Element(value, _next=self.head)
            self.head.prev = new_head
            self.head = new_head
    
    def __repr__(self):
        values = []
        cur = self.head
        while cur is not None:
            values.append(cur.value)
            cur = cur.next
        return repr(values)


In [88]:
from collections import deque

class FIFO3(deque):
    def push(self, value):
        self.append(value)
    
    def pop(self):
        try:
            return self.popleft()
        except IndexError:
            return None

In [89]:
def test_fifo1(queue):
    for x in numbers:
        queue.push(x)
    for _ in numbers:
        queue.pop()

def test_fifo2(queue):
    for x in numbers*100:
        queue.push(x)
    for _ in numbers*105:
        queue.pop()


In [90]:
queue1 = FIFO1()
queue2 = FIFO2()
queue3 = FIFO3()

print(timeit(lambda: test_fifo1(queue1), number=10000))
print(timeit(lambda: test_fifo1(queue2), number=10000))
print(timeit(lambda: test_fifo1(queue3), number=10000))

queue1 = FIFO1()
queue2 = FIFO2()
queue3 = FIFO3()

print(timeit(lambda: test_fifo2(queue1), number=10000))
print(timeit(lambda: test_fifo2(queue2), number=10000))
print(timeit(lambda: test_fifo2(queue3), number=10000))

0.018370100000538514
0.030239300000175717
0.011435200000050827
1.3911026999994647
2.696549799999957
0.9940397000000303


Заметно, что класс FIFO2 проигрывает классу FIFO1 в производительности так же как и в потреблении памяти. FIFO2 все же хранит каждое значение в обертке из класса `Element`, который в свою очередь помимо значения хранит ссылки на другие элементы списка.

Если же сравнивать реализации FIFO1 и FIFO3, то очевидно, что FIFO3 выигрывает по скорости работы. Связано это с тем, что FIFO3 написан на Си в то же время как FIFO1 написан на Python. Реализация FIFO2 - это попытка написать аналог FIFO3 на Python и как видно из тестов, она хоть и справляется с работой, но работает хуже реализации FIFO1. 

Важно отметить, что FIFO2 и FIFO3 используют двойной связанный список в то же время как FIFO1 хранит данные в динамическом массиве Python.

Получается, что двойной список работает эффективнее динамического массива однако реализация его на python малоэффективна и выгодней использовать реализацию на Си.

# Вопрос№3

In [93]:
def quicksort(values):
    if len(values) <= 1:
        return values
    pivot = values[len(values)//2]
    return quicksort(list(filter(lambda x: x < pivot, values))) + \
        list(filter(lambda x: x == pivot, values))+ \
        quicksort(list(filter(lambda x: x > pivot, values)))


In [94]:
quicksort([1,5, 6, 7, 8, 2, 4, 6, 1, 2, 3, 8 ,4])

[1, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8]

Выше представлена реализация классического алгоритма быстрой сортировки. Данный алгоритм сортирует массив быстрее всего в том смысле, что он обладает сложностью $O(n log_2 n)$.